# 빅카인드 웹크롤링

In [217]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.common.exceptions import NoSuchElementException

driver=webdriver.Chrome('C:/chromedriver')
driver.get('https://www.bigkinds.or.kr/')
driver.find_element_by_xpath('//*[@id="contents"]/div[3]/div/div[2]/div/div[2]/button').click()
driver.implicitly_wait(3)


### 로그인 과정

In [218]:
driver.find_element_by_xpath('//*[@id="app-top-navbar"]/ul/li[1]/a').click()
log_id = driver.find_element_by_id('login-user-id')
log_id.send_keys('webcrawl04@gmail.com')

log_pass = driver.find_element_by_id('login-user-password')
log_pass.send_keys('950628!!')
driver.find_element_by_id('login-btn').click()
time.sleep(1)

### 기간조정과정

In [219]:
driver.find_element_by_id('date-filter-btn').click()
begin_d = driver.find_element_by_id('search-begin-date')
begin_d.send_keys(Keys.HOME)
begin_d.send_keys('2015-01-01')
begin_d.send_keys(Keys.TAB)
end_d = driver.find_element_by_id('search-end-date')
end_d.send_keys(Keys.HOME)
end_d.send_keys('2015-12-31')
begin_d.send_keys(Keys.TAB)
driver.find_element_by_id('date-filter-btn').click()

### 검색어 입력과정

In [220]:
#검색어입력
search = driver.find_element_by_id('total-search-key')
search.send_keys('영재')
search.send_keys(Keys.RETURN)

In [221]:
#100건씩 보기
driver.find_element_by_xpath('//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select').click()
driver.find_element_by_xpath('//*[@id="collapse-step-2"]/div/div/div[2]/div[2]/div/div[2]/div[2]/select/option[4]').click()
time.sleep(0.5)

In [222]:
def crawl_nextpage(i=3):
    while True:
        address = '//*[@id="news-results-pagination"]/ul/li[' + f'{i}' + ']/a'
        i += 1
        try:
            driver.find_element_by_xpath(address).click()
            time.sleep(4)
        except NoSuchElementException:
            break;

In [223]:
while True:
    lst = driver.find_elements_by_css_selector('#news-results-pagination > ul > li')
    len(lst)
    print(len(lst))

    if len(lst)==10:
        crawl_nextpage()
        i=3
    else:
        crawl_nextpage()
        break;

10
10
10
10
10


ElementClickInterceptedException: Message: element click intercepted: Element <a href="#" class="page-link" data-page="...">25</a> is not clickable at point (568, 868). Other element would receive the click: <div class="news-item__body">...</div>
  (Session info: chrome=78.0.3904.97)


In [224]:
n_data

[]

## 자연어 처리

In [ ]:
import pandas as pd
import konlpy
from apyori import apriori
from itertools import chain
sbs=pd.DataFrame(sbs_news)
komoran=konlpy.tag.Komoran()

sbs_8=[]
for i,j in enumerate(sbs['8월']):
    words=(list(filter(lambda word: len(word)>1,
                       komoran.nouns('%r'%j))))
    sbs_8.append(words)

pd.DataFrame(sbs_8).to_csv('sbs_8.csv', encoding='utf-8', index=False)

In [ ]:
import pandas as pd
import konlpy
from apyori import apriori
import csv
with open('sbs_4.csv', 'r', encoding='utf-8') as cf:
    sbs_4_1=[]
    r=csv.reader(cf)
    for row in r:
        sbs_4_1.append(row)
rules=apriori(sbs_4_1, min_support=0.3, min_confidence=0.3)
result=list(rules)
result_df=pd.DataFrame(None, columns=['lhs','rhs','support','confidence','lift'])
index=0
for row in result:
    support=row[1]
    ordered_stat=row[2]
    for ordered_item in ordered_stat:
        lhs=' '.join(x.strip() for x in ordered_item[0])
        rhs=' '.join(x.strip() for x in ordered_item[1])
        confidence=ordered_item[2]
        lift=ordered_item[3]
        result_df.loc[index]=[lhs,rhs,support,confidence,lift]
        index=index+1
result_df.sort_values('lift',ascending=False)

In [ ]:
import pandas as pd
import konlpy
from apyori import apriori
import csv
with open('sbs_8.csv', 'r', encoding='utf-8') as cf:
    sbs_8_1=[]
    r=csv.reader(cf)
    for row in r:
        sbs_8_1.append(row)
rules=apriori(sbs_8_1, min_support=0.3, min_confidence=0.3)
result=list(rules)
result_df=pd.DataFrame(None, columns=['lhs','rhs','support','confidence','lift'])
index=0
for row in result:
    support=row[1]
    ordered_stat=row[2]
    for ordered_item in ordered_stat:
        lhs=' '.join(x.strip() for x in ordered_item[0])
        rhs=' '.join(x.strip() for x in ordered_item[1])
        confidence=ordered_item[2]
        lift=ordered_item[3]
        result_df.loc[index]=[lhs,rhs,support,confidence,lift]
        index=index+1
result_df.sort_values('lift',ascending=False)

In [ ]:
result_df.loc[(result_df.lhs.str.contains('일본'))&
             (result_df.rhs=='규제')].sort_values(by=['lift'],
                                               ascending=False)